In [1]:
!pip install parlai

In [2]:
import parlai

In [3]:
!pip install -q subword_nmt # extra requirement we need for this tutorial

In [4]:
from parlai.scripts.interactive import Interactive
Interactive.main(model_file='zoo:dodecadialogue/empathetic_dialogues_ft/model')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


17:01:01 | Overriding opt["model_file"] to /usr/local/lib/python3.8/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model (previously: data/models/dodecadialogue/empathetic_dialogues/model)
17:01:01 | Loading model with `--beam-block-full-context false`
17:01:01 | loading dictionary from /usr/local/lib/python3.8/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model.dict
17:01:02 | num words = 54946
17:01:02 | ImageSeq2seq: full interactive mode on.
17:01:02 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
17:01:03 | Total parameters: 88,559,104 (88,559,104 trainable)
17:01:03 | Loading existing model params from /usr/local/lib/python3.8/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model
17:01:06 | Opt:
17:01:06 |     activation: gelu
17:01:06 |     adafactor_eps: '(1e-30, 0.001)'
17:01:06 |     adam_eps: 1e-08
17:01:06 |     add_p1_after_newln: False
17:01:06 |     

In [5]:
# The display_data script is used to show the contents of a particular task.
# By default, we show the train
from parlai.scripts.display_data import DisplayData
DisplayData.main(task='empathetic_dialogues', num_examples=5)

17:01:51 | Opt:
17:01:51 |     allow_missing_init_opts: False
17:01:51 |     batchsize: 1
17:01:51 |     datapath: /usr/local/lib/python3.8/dist-packages/data
17:01:51 |     datatype: train:ordered
17:01:51 |     dict_class: None
17:01:51 |     display_add_fields: 
17:01:51 |     download_path: None
17:01:51 |     dynamic_batching: None
17:01:51 |     hide_labels: False
17:01:51 |     ignore_agent_reply: True
17:01:51 |     image_cropsize: 224
17:01:51 |     image_mode: raw
17:01:51 |     image_size: 256
17:01:51 |     init_model: None
17:01:51 |     init_opt: None
17:01:52 |     is_debug: False
17:01:52 |     loglevel: info
17:01:52 |     max_display_len: 1000
17:01:52 |     model: None
17:01:52 |     model_file: None
17:01:52 |     multitask_weights: [1]
17:01:52 |     mutators: None
17:01:52 |     num_examples: 5
17:01:52 |     override: "{'task': 'empathetic_dialogues', 'num_examples': 5}"
17:01:52 |     parlai_home: /usr/local/lib/python3.8/dist-packages
17:01:52 |     remove_poli

# Training a model

Well it's one thing looking at data, but what if we want to train our own model (from scratch)? Let's train a very simple transformer/generator with attention, to respond to blended_skill talk dialogues.



In [10]:
!rm -rf from_pretrained
!mkdir -p from_pretrained
from parlai.scripts.train_model import TrainModel
TrainModel.main(
    # similar to before
    task='blended_skill_talk', 
    model='transformer/generator',
    model_file='from_pretrained/model',
    
    # initialize with a pretrained model
    init_model='zoo:blender/blender_90M/model',
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,model_parallel=True,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
 dict_file='/usr/local/lib/python3.8/dist-packages/data/models/blender/blender_90M/model.dict',
    learn_positional_embeddings=True,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adamax',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 60 minutes/1 Hour, and validate every 0.25 epochs
    max_train_time=3600, validation_every_n_epochs=0.25,
    
    # depend on our system. Synce we have pretty good AI accelerator it works fine
    batchsize=24, fp16=True, fp16_impl='safe',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our accelerator at a time
    dynamic_batching='full',
)

17:23:44 | building dictionary first...
17:23:44 | No model with opt yet at: from_pretrained/model(.opt)
17:23:44 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,is_debug: False,datapath: /usr/local/lib/python3.8/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: True,checkpoint_activations: False,beam_block_full_context: True,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,interactive_mode: False,history_reversed: False,history_add_global_end_token: None,special_tok_lst: None,bpe_vocab: None,bpe_merge: None,bpe_add_

({'exs': SumMetric(5651),
  'clen': AverageMetric(162.7),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'llen': AverageMetric(18.14),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'loss': AverageMetric(2.675),
  'ppl': PPLMetric(14.52),
  'token_acc': AverageMetric(0.4298),
  'token_em': AverageMetric(0.000177),
  'exps': GlobalTimerMetric(126),
  'ltpb': GlobalAverageMetric(1282),
  'ltps': GlobalTimerMetric(2285),
  'ctpb': GlobalAverageMetric(1.149e+04),
  'ctps': GlobalTimerMetric(2.049e+04),
  'tpb': GlobalAverageMetric(1.277e+04),
  'tps': GlobalTimerMetric(2.277e+04),
  'lr': GlobalAverageMetric(8.3e-06),
  'total_train_updates': GlobalFixedMetric(83)},
 {'exs': SumMetric(5482),
  'clen': AverageMetric(165),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'llen': AverageMetric(18.88),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'loss': AverageMetric(2.669),
  'ppl': PPLMetric(14.42),
  'token_acc': Avera

In [13]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='blended_skill_talk',
    model_file='from_pretrained/model',
    num_examples=2,  skip_generation=False,
)

18:21:58 | Overriding opt["skip_generation"] to False (previously: True)
18:21:58 | loading dictionary from from_pretrained/model.dict
18:21:58 | num words = 54944
18:21:59 | Total parameters: 87,508,992 (87,508,992 trainable)
18:21:59 | Loading existing model params from from_pretrained/model
18:22:00 | creating task(s): blended_skill_talk
18:22:00 | Loading ParlAI text data: /usr/local/lib/python3.8/dist-packages/data/blended_skill_talk/valid.txt
18:22:01 | Opt:
18:22:01 |     activation: gelu
18:22:01 |     adafactor_eps: '[1e-30, 0.001]'
18:22:01 |     adam_eps: 1e-08
18:22:01 |     add_p1_after_newln: False
18:22:01 |     aggregate_micro: False
18:22:01 |     allow_missing_init_opts: False
18:22:01 |     attention_dropout: 0.0
18:22:01 |     batchsize: 24
18:22:01 |     beam_block_full_context: True
18:22:01 |     beam_block_list_filename: None
18:22:01 |     beam_block_ngram: -1
18:22:01 |     beam_context_block_ngram: -1
18:22:01 |     beam_delay: 30
18:22:01 |     beam_length_p

In [15]:
from parlai.scripts.interactive import Interactive
Interactive.main(model_file='from_pretrained/model', task='blended_skill_talk', beam_block_ngram= 3, beam_context_block_ngram= 3)

18:23:56 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
18:23:56 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
18:23:56 | loading dictionary from from_pretrained/model.dict
18:23:56 | num words = 54944
18:23:56 | TransformerGenerator: full interactive mode on.
18:23:57 | Total parameters: 87,508,992 (87,508,992 trainable)
18:23:57 | Loading existing model params from from_pretrained/model
18:23:57 | Opt:
18:23:57 |     activation: gelu
18:23:57 |     adafactor_eps: '[1e-30, 0.001]'
18:23:57 |     adam_eps: 1e-08
18:23:57 |     add_p1_after_newln: False
18:23:57 |     aggregate_micro: False
18:23:57 |     allow_missing_init_opts: False
18:23:57 |     attention_dropout: 0.0
18:23:57 |     batchsize: 24
18:23:57 |     beam_block_full_context: True
18:23:57 |     beam_block_list_filename: None
18:23:57 |     beam_block_ngram: 3
18:23:57 |     beam_context_block_ngram: 3
18:23:57 |     beam_delay: 30
18:23:57 |     beam_length_penalty: 0.65
18:23:57 |     be

## Train on custom dataset

In [2]:
from parlai.core.teachers import register_teacher, DialogTeacher
from parlai.scripts.display_model import DisplayModel
from parlai.scripts.train_model import TrainModel
from parlai.scripts.display_data import DisplayData
from parlai.core.agents import register_agent, Agent
file = open('data.txt')
lines = file.readlines()

odd_lines = []
even_lines = []

for i,line in enumerate(lines):
    if i%2==0:
        even_lines.append(line.strip('\n'))
    else:
        odd_lines.append(line.strip('\n'))
from parlai.core.teachers import register_teacher, DialogTeacher
!rm -rf out.txt

@register_teacher("my_teacher")
class MyTeacher(DialogTeacher):
    def __init__(self, opt, shared=None):
        # opt is the command line arguments.
        
        # What is this shared thing?
        # We make many copies of a teacher, one-per-batchsize. Shared lets us store 
        
        # We just need to set the "datafile".  This is boilerplate, but differs in many teachers.
        # The "datafile" is the filename where we will load the data from. In this case, we'll set it to
        # the fold name (train/valid/test) + ".txt"
        opt['datafile'] = opt['datatype'].split(':')[0] + ".txt"
        super().__init__(opt, shared)
    
    def setup_data(self, datafile):
        # filename tells us where to load from.
        # We'll just use some hardcoded data, but show how you could read the filename here:
        print(f" ~~ Loading from {datafile} ~~ ")
      
        with open('out.txt','a') as out:

          for e,o in zip(even_lines,odd_lines):
            yield (e,o), True

              #out.write(e+"," + o+'\n')
                
           
      
        
        
DisplayData.main(task="my_teacher")

19:12:19 | Opt:
19:12:19 |     allow_missing_init_opts: False
19:12:19 |     batchsize: 1
19:12:19 |     datapath: /usr/local/lib/python3.8/dist-packages/data
19:12:19 |     datatype: train:ordered
19:12:19 |     dict_class: None
19:12:19 |     display_add_fields: 
19:12:19 |     download_path: None
19:12:19 |     dynamic_batching: None
19:12:19 |     hide_labels: False
19:12:19 |     ignore_agent_reply: True
19:12:19 |     image_cropsize: 224
19:12:19 |     image_mode: raw
19:12:19 |     image_size: 256
19:12:19 |     init_model: None
19:12:19 |     init_opt: None
19:12:19 |     is_debug: False
19:12:19 |     loglevel: info
19:12:19 |     max_display_len: 1000
19:12:19 |     model: None
19:12:19 |     model_file: None
19:12:19 |     multitask_weights: [1]
19:12:19 |     mutators: None
19:12:19 |     num_examples: 10
19:12:19 |     override: "{'task': 'my_teacher'}"
19:12:19 |     parlai_home: /usr/local/lib/python3.8/dist-packages
19:12:19 |     starttime: Feb28_19-12
19:12:19 |     t

In [3]:
!rm -rf from_custom_trained
!mkdir -p from_custom_trained
from parlai.scripts.train_model import TrainModel
TrainModel.main(
    # similar to before
    task='my_teacher', 
    model='transformer/generator',
    model_file='from_custom_trained/model',
    
    # initialize with a pretrained model
    init_model='zoo:blender/blender_90M/model',
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,model_parallel=True,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
 dict_file='/usr/local/lib/python3.8/dist-packages/data/models/blender/blender_90M/model.dict',
    learn_positional_embeddings=True,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adamax',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    max_train_time=600, validation_every_n_epochs=0.25,
    
    # depend on our system. Synce we have pretty good AI accelerator it works fine
    batchsize=24, fp16=True, fp16_impl='safe',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our accelerator at a time
    dynamic_batching='full',
)

19:14:11 | building dictionary first...
19:14:11 | No model with opt yet at: from_custom_trained/model(.opt)
19:14:11 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,is_debug: False,datapath: /usr/local/lib/python3.8/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: True,checkpoint_activations: False,beam_block_full_context: True,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,interactive_mode: False,history_reversed: False,history_add_global_end_token: None,special_tok_lst: None,bpe_vocab: None,bpe_merge: None,bpe_

({'exs': SumMetric(4),
  'clen': AverageMetric(19.5),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'llen': AverageMetric(35.75),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'loss': AverageMetric(1.514),
  'ppl': PPLMetric(4.545),
  'token_acc': AverageMetric(0.6853),
  'token_em': AverageMetric(0),
  'exps': GlobalTimerMetric(45.83),
  'ltpb': GlobalAverageMetric(143),
  'ltps': GlobalTimerMetric(1639),
  'ctpb': GlobalAverageMetric(78),
  'ctps': GlobalTimerMetric(894.3),
  'tpb': GlobalAverageMetric(221),
  'tps': GlobalTimerMetric(2534),
  'lr': GlobalAverageMetric(7.5e-06),
  'total_train_updates': GlobalFixedMetric(75)},
 {'exs': SumMetric(4),
  'clen': AverageMetric(19.5),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'llen': AverageMetric(35.75),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'loss': AverageMetric(1.514),
  'ppl': PPLMetric(4.545),
  'token_acc': AverageMetric(0.6853),
  'token_em': Av

In [5]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='blended_skill_talk',
    model_file='from_custom_trained/model',
    num_examples=2,  skip_generation=False,
)

19:40:30 | Overriding opt["task"] to blended_skill_talk (previously: my_teacher)
19:40:30 | Overriding opt["skip_generation"] to False (previously: True)
19:40:30 | loading dictionary from from_custom_trained/model.dict
19:40:30 | num words = 54944
19:40:31 | Total parameters: 87,508,992 (87,508,992 trainable)
19:40:31 | Loading existing model params from from_custom_trained/model
19:40:32 | creating task(s): blended_skill_talk
19:40:32 | Loading ParlAI text data: /usr/local/lib/python3.8/dist-packages/data/blended_skill_talk/valid.txt
19:40:32 | Opt:
19:40:32 |     activation: gelu
19:40:32 |     adafactor_eps: '[1e-30, 0.001]'
19:40:32 |     adam_eps: 1e-08
19:40:32 |     add_p1_after_newln: False
19:40:32 |     aggregate_micro: False
19:40:32 |     allow_missing_init_opts: False
19:40:32 |     attention_dropout: 0.0
19:40:32 |     batchsize: 24
19:40:32 |     beam_block_full_context: True
19:40:32 |     beam_block_list_filename: None
19:40:32 |     beam_block_ngram: -1
19:40:32 |   

In [6]:
from parlai.scripts.interactive import Interactive
Interactive.main(model_file='from_custom_trained/model', task='blended_skill_talk', beam_block_ngram= 3, beam_context_block_ngram= 3)

19:40:44 | Overriding opt["task"] to blended_skill_talk (previously: my_teacher)
19:40:44 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
19:40:44 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
19:40:44 | loading dictionary from from_custom_trained/model.dict
19:40:44 | num words = 54944
19:40:44 | TransformerGenerator: full interactive mode on.
19:40:45 | Total parameters: 87,508,992 (87,508,992 trainable)
19:40:45 | Loading existing model params from from_custom_trained/model
19:40:46 | Opt:
19:40:46 |     activation: gelu
19:40:46 |     adafactor_eps: '[1e-30, 0.001]'
19:40:46 |     adam_eps: 1e-08
19:40:46 |     add_p1_after_newln: False
19:40:46 |     aggregate_micro: False
19:40:46 |     allow_missing_init_opts: False
19:40:46 |     attention_dropout: 0.0
19:40:46 |     batchsize: 24
19:40:46 |     beam_block_full_context: True
19:40:46 |     beam_block_list_filename: None
19:40:46 |     beam_block_ngram: 3
19:40:46 |     beam_context_block_ngram: 3

KeyboardInterrupt: Interrupted by user